#### 将文件夹压缩到zip文件，并复制到剪贴板

In [15]:
import win32clipboard
from ctypes import *
import zipfile
import datetime
import os

class DROPFILES(Structure):
    _fields_ = [
        ("pFiles", c_uint),
        ("x", c_long),
        ("y", c_long),
        ("fNC", c_int),
        ("fWide", c_bool),
    ]

pDropFiles = DROPFILES()
pDropFiles.pFiles = sizeof(DROPFILES)
pDropFiles.fWide = True
matedata = bytes(pDropFiles)

def set_clipboard_files(paths):
    files = ("\0".join(paths)).replace("/", "\\")
    data = files.encode("U16")[2:]+b"\0\0"
    win32clipboard.OpenClipboard()
    try:
        win32clipboard.EmptyClipboard()
        win32clipboard.SetClipboardData(
            win32clipboard.CF_HDROP, matedata+data)
    finally:
        win32clipboard.CloseClipboard()

# 复制指定文件到剪切板
def set_clipboard_file(file):
    set_clipboard_files([file])

# 读取剪切板中的文件路径
def read_clipboard_file_paths():
    win32clipboard.OpenClipboard()
    paths = None
    try:
        return win32clipboard.GetClipboardData(win32clipboard.CF_HDROP)
    finally:
        win32clipboard.CloseClipboard()

# 压缩文件夹并将结果复制到剪切板
# project_path: ../project
def zip_folder_and_copy(project_path, more_info):
    # 获取当前日期
    today = datetime.datetime.now().strftime('%y%m%d')
    time = datetime.datetime.now().strftime('%H%M')

    if more_info == '':
        suffix = today + '_' + time + '.zip'
    else:
        suffix = today + '_' + more_info + '.zip'

    input_path = project_path + '/dist'
    output_path = project_path + '/release/prod_' + suffix

    # 压缩文件夹
    zip = zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED)
    for path, dir_names, filenames in os.walk(input_path):
        fpath = path.replace(input_path, '')
        for filename in filenames:
            zip.write(os.path.join(path, filename), os.path.join(fpath, filename))
    zip.close()

    # 将压缩得到的zip文件复制到剪切板
    set_clipboard_file(output_path)